# Fields



**Key concepts:**
* {py:class}`~coordax.Field` is Coordax's core data structure, an array with optional labeled axes.
* `Field` methods {py:meth}`~coordax.Field.tag` and {py:meth}`~coordax.Field.untag` are used to attach/remove coordinates
* Array dimensions labeled with `None` correspond to locally *positional* axes
* Labeled dimensions facilitate reordering and broadcasting of underlying data
* Coordinates are checked for consistency to catch alignment errors

## Instantiating `Field`

To instantiate a {py:class}`~cx.Field` object, either use the `Field` constructor directly (`Field(data: Array, dims: tuple[str, ...], ...)`) or the {py:func}`~coordax.wrap` helper function (`cx.wrap(data: Array, *dims: str | Coordinate)`):

In [ ]:
import coordax as cx
import jax
import jax.numpy as jnp
import numpy as np

field_data = np.random.RandomState(0).uniform(size=(3, 4))

field = cx.wrap(field_data, 'x', 'y')
field

The core `Field` attributes are `data`, `dims` and `axes`:

In [ ]:
field.data

In [ ]:
field.dims

To specify an axis of a `Field` as unlabeled, use `None` rather than a string.
Alternatively, you can use {py:class}`~cx.Coordinate` objects to annotate axes (e.g., to add tick labels), which we'll cover in detail [later](coordinates.ipynb).

In [ ]:
partially_labeled_field = cx.wrap(field_data, 'x', None)
partially_labeled_field

When some of the axes of a `Field` are not labeled, we say that the Field has positional axes. Positional axes play a crucial role in how computation is performed on `Field` objects.

The overall shape of a `Field` is partitioned into `positional_shape` and `named_shape` properties, indicating the sizes of axes associated with each.

In [ ]:
print(f'{field.shape=}')
print(f'{field.positional_shape=}')
print(f'{field.named_shape=}')

print(f'{partially_labeled_field.shape=}')
print(f'{partially_labeled_field.positional_shape=}')
print(f'{partially_labeled_field.named_shape=}')

## Updating coordinate labels using `tag/untag`

To instantiate {py.class}`~cx.Field` with a different set of coordinate labels from another `Field`, it is convenient to use `tag/untag` methods.

* {py:meth}`cx.Field.tag` can be used on a `Field` to label all existing positional axes
* {py:meth}`cx.Field.tag` can be used on a fully labeled `Field` to make provided dimensions positions

These restrictions are necessary to avoid ambiguity in which positional axis is being tagged or in which order should untagged axes appear in the `positional_shape`. Enforcing the above rules removes such ambiguity.

In [ ]:
partially_labeled_field = field.untag('y')
labeled_field = partially_labeled_field.tag('y')  # same as field
cx.testing.assert_fields_equal(field, labeled_field)

When dealing with a pytee of `Field` objects, it may be desired to relabel all entries. This can be done using simple helpers `cx.tag` and `cx.untag`.

In [ ]:
tree = {
    'a': cx.wrap(np.ones((1, 1, 1)), 'x', 'y', 'z'),
    'tuple': (cx.wrap(np.array([np.pi]), 'x'), cx.wrap(np.array([np.e]), 'x')),
}
tree

In [ ]:
cx.untag(tree, 'x')

In [ ]:
cx.tag(cx.untag(tree, 'x'), 'batch')

Later we will see that a common pattern with `coordax` is to:
1. untag dimension on which we want to operate
2. perform the desired computation
3. retag the result with approriate coordinates

## Reordering, broadcasting and basic operations on `Field`

Reordering and broadcasting of `Field` objects with {py:meth}`~coordax.Field.order_as` and {py:meth}`~coordax.Field.broadcast_like` can be done using their coordinates or dimension names:

In [ ]:
f_xy = cx.wrap(np.arange(2 * 3).reshape((2, 3)), 'x', 'y')
f_yx = f_xy.order_as('y', 'x')  # transpose to a new order.
print(f'{f_xy.dims=}, {f_yx.dims=}')

In [ ]:
f = cx.wrap(np.ones((4, 7)), 'x', 'y')
arange = cx.wrap(np.arange(4), 'x')
b_arange = arange.broadcast_like(f)  # broadcast to another
b_arange

Ellipsis can also be used in `tag()` and `order_as()` to indicate "all other dimensions:"

In [ ]:
f = cx.wrap(np.ones((1, 2, 4, 3)), 'x', 'y', 'z', 'q')
f_qy = f.order_as('q', 'y', ...)
f_xq = f.untag('y', 'z')
another_f_xq = cx.wrap(np.ones((1, 2, 4, 3)), 'x', ..., 'q')
assert another_f_xq.dims == f_xq.dims
f_qy, f_xq

`Field` also supports a small set of operations directly, including arithmetic with python numeric types (e.g., scaling):

In [ ]:
cx.wrap(np.arange(5), 'x') * 2